In [55]:
import torch

In [56]:
inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your
    [0.55, 0.87, 0.66],  # journey
    [0.57, 0.85, 0.64],  # starts
    [0.22, 0.58, 0.33],  # with
    [0.77, 0.25, 0.10],  # one
    [0.05, 0.80, 0.55]]  # step
)

In [57]:
query = inputs[1] # segundo token é a query
attn_scores = torch.empty(inputs.shape[0])

for i, x_i in enumerate(inputs):
    attn_scores[i] = torch.dot(x_i, query) # produto escalar para descobrir similaridade (alinhamento)

print(attn_scores)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [58]:
# Normalização serve para interpretação e estabilidade no treino, passar
# para formato de porcentagens
# Após normalização, temos o tensor attention weights
# Principal objetivo fazer com que a soma dos attention weights seja 1
attn_weights_tmp = attn_scores / attn_scores.sum() # Jeito mais simples é dividir cada um pela soma
print(attn_weights_tmp)

tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])


In [59]:
# No entanto, é recomendado usar a função softmax para normalização
# Quando temos valores extremos, é ideal que valores pequenos sejam 0
# e valores grandes sejam 1, para não confundir o otimizador durante 
# back propagation, pois ele daria importância para esses pequenos valores

# Sendo xn um attention score qualquer
# Softmax tem uma fórmula e^xn / sum, sendo sum = e^xi + e^xi+1...
# Pytorch tem uma implementação e^(xn - max), sendo max o valor máximo
attn_weights = torch.softmax(attn_scores, dim=0)
print("Attention weights: ", attn_weights)

Attention weights:  tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])


In [60]:
query = inputs[1] # tensor de journey
context_vec = torch.zeros(query.shape) # iniciando tensor com valores zero

for i, x_i in enumerate(inputs):
    context_vec += attn_weights[i] * x_i

print(context_vec)

tensor([0.4419, 0.6515, 0.5683])


In [61]:
# Computando attention scores para cada query
attn_scores = torch.empty(6, 6)

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)
        
print(attn_scores) # matriz de attention scores

# A matriz resulutante é do tipo:

#              Journey     starts      with    ...
#
#    Journey    
#     starts
#       with
#       ...

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [62]:
# Como for loops são lentos, usaremos a multiplicação de matrizes do pytorch
# Produto escalar faz operações de linhas por colunas, então multiplicarmos
# a matriz original pela transposta tem o mesmo efeito e é mais rápido
attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [63]:
# Normalização
attn_weights = torch.softmax(attn_scores, dim=-1) # dim especifica a dimensão do tensor de input 
# Com dim = -1 instruímos softmax a aplicar a normalização na última dimensão do tensor attn_scores (a última dimensão são as colunas)
print("Attention weights: ", attn_weights) # todas as linhas e colunas somam 1

Attention weights:  tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [64]:
# Calculando os context vectors
context_vecs = attn_weights @ inputs
print(context_vecs)

# O cálculo de uma linha também pode ser definido como
# context_vec21 = attn_weights21 * inputs[0] (vetor da linha 1)
# context_vec22 = attn_weights22 * inputs[1] (vetor da linha 2)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])
